In [1]:
import collections

def l():
    return [int(x) for x in open("13.txt").read().split(',')]


class m:
    idgen=0
    
    #get op and modes of following 3 addresses
    def ix(s,x):
        assert(x>=0)
        x="%05d"%(x)
        op=int(x[3:])
        m3,m2,m1=[int(c) for c in x[:3]]
        return op,m1,m2,m3


    def gv(s,p,ip,m,ix):
        assert(ix in [1,2,3])
        if m==0:#reg
            return p[p[ip+ix]]
        elif m==1:#imm
            return p[ip+ix] 
        elif m==2:#reg+rb
            return p[p[ip+ix]+s.rb]
        assert(0)
        
            
        

    
    def __init__(s,p):
        #load to mem
        s.p=collections.defaultdict(int)
        for i,v in enumerate(p):
            s.p[i]=v
            
        #possible data source
        s.supplier=None
        
        #input queue
        s.i=[]
        
        #id
        s.id=m.idgen
        m.idgen+=1
        
        #iterator for execution
        s.it=s.iterator()
        
        #print("M",s.id,"created")
        
        #relative base
        s.rb=0
        
    
    def sendInput(s, i):
        s.i.append(i)
        
    def getData(s):
        return next(s.it)
        
    def iterator(s):
        ip=0
        c=0
        while 1:
            op,m1,m2,m3=s.ix(s.p[ip])
            
            if op==1:#add
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)+s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==2:#mul
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)*s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==3:#in
                assert(m1 in [0,2])
                r=None
                if len(s.i)>0:
                    r=s.i[0]
                    s.i=s.i[1:]
                else:
                    if s.supplier:
                        r=s.supplier.getData()
                    else:
                        print("id",s.id, "returning zero")
                        #assert(0)
                        r=0
                tix=s.p[ip+1]
                if m1==2:
                    tix+=s.rb
                s.p[tix]=r
                ip+=2
            elif op==4:#out
                
                r=s.gv(s.p,ip,m1,1)
                #print(s.id,"spits out",r)
                yield r
                #print("--id:",s.id,"onwards")
                ip+=2
                
            elif op==5:#jump-if-true
                if s.gv(s.p,ip,m1,1):
                    ip=s.gv(s.p,ip,m2,2)
                else:
                    ip+=3
            elif op==6:#jump-if-false
                if not s.gv(s.p,ip,m1,1):
                    ip=s.gv(s.p,ip,m2,2)
                else:
                    ip+=3
            elif op==7:#less-than
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)<s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==8:#equals
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)==s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==9:#adjust relative base
                s.rb+=s.gv(s.p,ip,m1,1)
                ip+=2
            elif op in [99]:#end
                #print("fin", s.id)
                break
            else:
                print("id", s.id, "op",op,"c",c)
                assert(0)
            #print(op)
            c+=1
        
        yield None

In [2]:
"""

0 is an empty tile. No game object appears in this tile.
1 is a wall tile. Walls are indestructible barriers.
2 is a block tile. Blocks can be broken by the ball.
3 is a horizontal paddle tile. The paddle is indestructible.
4 is a ball tile. The ball moves diagonally and bounces off objects.

"""

labels=["empty","wall","block","hpt","ball"]

class r:
    def __init__(s):
        
        s.m=m(l())
        s.m.supplier=s
        s.mp=collections.defaultdict(int)
        s.score=0
        s.batx=None
        s.ballx=None
    
    def upd(s):
        x=s.m.getData()
        if x is None:
            return 0
        y=s.m.getData()
        if y is None:
            return 0
        t=s.m.getData()
        if t is None:
            return 0
        if x==-1 and y==0:
            s.score=t
            #print(t)
        else:
            s.mp[(x,y)]=t
            if t==4:
                s.ballx=x
            elif t==3:
                s.batx=x
        
        return 1
     
    
    
    def getData(s):
        if s.ballx<s.batx:
            return -1
        elif s.ballx>s.batx:
            return 1
        return 0
        
def p1():
    b=r()
    while b.upd():
        pass
    return sum([1 if x==2 else 0 for x in b.mp.values()])

def p2():
    b=r()
    b.m.p[0]=2
    while b.upd():
        pass
    return b.score

print("part1:", p1(),412)
print("part2:", p2(),20940)



part1: 412 412
part2: 20940 20940
